In [2]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
# 데이터 생성 함수
def generate_data(num_samples):
    X = np.random.randint(0, 10, (num_samples, 5))
    y = np.flip(X, axis = 1)
    return X, y

In [4]:
# 데이터 생성
num_samples = 20000
X, y = generate_data(num_samples)

In [ ]:
# 데이터 형태 변환 (LSTM을 사용하기 위해 3D 텐서로 변환)
X = X.reshape((X.shape[0], X.shape[1], 1))
y = y.reshape((y.shape[0], y.shape[1], 1))

In [5]:
# 인코더 정의
encoder_inputs = Input(shape = (5, 1))
encoder = LSTM(128, return_state = True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [6]:
# 디코더 정의
decoder_inputs = Input(shape = (5, 1))
decoder_lstm = LSTM(128, return_sequences = True, return_state = True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)
decoder_dense = Dense(10, activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [8]:
# 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [9]:
# 입력 데이터와 타겟 데이터를 동일하게 맞춤
decoder_input_data = np.zeros_like(X)

In [10]:
# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)

In [11]:
# 모델 학습
model.fit([X, decoder_input_data], y, epochs = 50, validation_split = 0.2, callbacks = [early_stopping])

Epoch 1/50
500/500 [==============================] - 13s 19ms/step - loss: 1.6562 - accuracy: 0.3648 - val_loss: 0.9247 - val_accuracy: 0.7520
Epoch 2/50
500/500 [==============================] - 8s 17ms/step - loss: 0.6207 - accuracy: 0.9111 - val_loss: 0.3893 - val_accuracy: 0.9811
Epoch 3/50
500/500 [==============================] - 6s 13ms/step - loss: 0.2596 - accuracy: 0.9901 - val_loss: 0.1471 - val_accuracy: 0.9998
Epoch 4/50
500/500 [==============================] - 8s 16ms/step - loss: 0.1102 - accuracy: 0.9948 - val_loss: 0.0637 - val_accuracy: 0.9999
Epoch 5/50
500/500 [==============================] - 8s 16ms/step - loss: 0.0801 - accuracy: 0.9883 - val_loss: 0.0352 - val_accuracy: 1.0000
Epoch 6/50
500/500 [==============================] - 7s 15ms/step - loss: 0.0263 - accuracy: 1.0000 - val_loss: 0.0204 - val_accuracy: 1.0000
Epoch 7/50
500/500 [==============================] - 7s 14ms/step - loss: 0.0486 - accuracy: 0.9893 - val_loss: 0.0609 - val_accuracy: 0.986

In [12]:
# 모델 검증 함수
def predict_reverse(input_sequence):
    input_sequence = np.array(input_sequence).reshape((1, 5, 1))
    decoder_input = np.zeros((1, 5, 1))
    predicted_sequence =model.predict([input_sequence, decoder_input])
    return np.argmax(predicted_sequence, axis = -1).reshape((5, ))

In [13]:
# 검증용 데이터 생성
test_X, test_y = generate_data(20)

In [14]:
# 모델 예측 및 정확도 계산
correct_predictions = 0
total_predictions = len(test_X)

for i in range(total_predictions):
    input_sequence = test_X[i]
    true_output = test_y[i].reshape(5, )
    predicted_output = predict_reverse(input_sequence)

    if np.array_equal(predicted_output, true_output):
        correct_predictions += 1

    print(f"입력: {input_sequence.reshape(5, )}")
    print(f"예측된 출력: {predicted_output}")
    print(f"실제 출력: {true_output}")
    print(f"정확 여부: {'맞음' if np.array_equal(predicted_output, true_output) else '틀림'}\n")

accuracy = correct_predictions / total_predictions
print(f"총 정확도: {accuracy * 100:.2f}%")

1/1 [==============================] - 1s 1s/step
입력: [8 4 5 6 2]
예측된 출력: [2 6 5 4 8]
실제 출력: [2 6 5 4 8]
정확 여부: 맞음

1/1 [==============================] - 0s 36ms/step
입력: [8 8 7 2 3]
예측된 출력: [3 2 7 8 8]
실제 출력: [3 2 7 8 8]
정확 여부: 맞음

1/1 [==============================] - 0s 41ms/step
입력: [5 5 0 4 9]
예측된 출력: [9 4 0 5 5]
실제 출력: [9 4 0 5 5]
정확 여부: 맞음

1/1 [==============================] - 0s 53ms/step
입력: [9 2 2 5 1]
예측된 출력: [1 5 2 2 9]
실제 출력: [1 5 2 2 9]
정확 여부: 맞음

1/1 [==============================] - 0s 64ms/step
입력: [2 7 5 0 3]
예측된 출력: [3 0 5 7 2]
실제 출력: [3 0 5 7 2]
정확 여부: 맞음

1/1 [==============================] - 0s 58ms/step
입력: [8 9 1 3 8]
예측된 출력: [8 3 1 9 8]
실제 출력: [8 3 1 9 8]
정확 여부: 맞음

1/1 [==============================] - 0s 50ms/step
입력: [8 7 7 8 7]
예측된 출력: [7 8 7 7 8]
실제 출력: [7 8 7 7 8]
정확 여부: 맞음

1/1 [==============================] - 0s 46ms/step
입력: [2 0 2 6 7]
예측된 출력: [7 6 2 0 2]
실제 출력: [7 6 2 0 2]
정확 여부: 맞음

1/1 [==============================] - 0s 48ms/step
입력: [4